In [4]:
import os
import sys
import keras
import tensorflow as tf
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/packages')
    !pip install git+https://github.com/keras-team/keras-tuner.git
    !pip install autokeras
    config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} )
    sess = tf.Session(config=config)
    keras.backend.set_session(sess)
else:
    path_fill = 'data/2022/filled_trian.csv'
    path_raw_train='data/2022/train.csv'
    path_A='data/2022/test_A榜.csv'
    path_B='data/2022/test_B榜.csv'

In [5]:
import pandas as pd
import useful_functions as uf
import autokeras as ak
df_train = pd.read_csv(path_raw_train).head(500)
df_train_filled = pd.read_csv(path_fill,index_col=0).head(500)
df_test_A = pd.read_csv(path_A)


Mode first,then go IQR filtering, then OneHot, then check corrlation, then

IQR method

Many has only zero, delete one with zero

In [6]:
df_nu = uf.get_numerical_df(df_train_filled)

In [7]:
IQR_filtering = uf.IQR_filtering

IQR filtering

In [8]:
df_train_filled.loc[:,IQR_filtering(df_nu).columns] = IQR_filtering(df_nu)

OneHot create_dummies()

In [9]:
df_train_filled=uf.create_dummies(df_train_filled)

now drop labels
uf.drop_high_corr_columns()---->list

In [10]:
x_sample = df_train_filled.drop(['CUST_UID'], axis=1)
x_sample.sample(frac=1).reset_index(drop=True)
y_sample = x_sample.LABEL
x_sample = x_sample.drop(['LABEL'], axis=1)

In [11]:
drop_col = uf.drop_high_corr_columns(df_train, x_sample)
x_sample=x_sample.drop(drop_col, axis=1)

看一下分布

In [12]:
if False: # 导入必要的库
    import matplotlib.pyplot as plt

    # 创建一个示例dataframe
    df = pd.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [6, 7, 8, 9, 10], 'C': [11, 12, 13, 14, 15]})

    # 调整图形大小
    plt.figure(figsize=(30, 25))

    # 绘制每一列的直方图
    x_sample.iloc[:, :43].hist(legend=None)


    # 调整子图之间的间距
    plt.tight_layout()

    # 显示图形
    plt.show()

In [13]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=3
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    # The path to the train.csv file.
    x=x_sample,
    y=y_sample,
    validation_split=0.15,
    epochs=10,
)


Trial 3 Complete [00h 00m 34s]
val_accuracy: 0.557692289352417

Best val_accuracy So Far: 0.75
Total elapsed time: 00h 02m 12s
INFO:tensorflow:Oracle triggered exit


2023-04-27 16:26:45.938598: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_34' with dtype int64 and shape [500]
	 [[{{node Placeholder/_34}}]]
2023-04-27 16:26:45.939456: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_47' with dtype int64 and shape [500]
	 [[{{node Placeholder/_47}}]]
2023-04-27 16:26:46.028453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-27 16:26:46.133907: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate a

16/16 [==============================] - 6s 166ms/step - loss: 0.5621 - accuracy: 0.7500


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [15]:
# Predict with the best model.
# predicted_y = clf.predict(df_test_A)
# Evaluate the best model with testing data.
print(clf.evaluate(x_sample,y_sample))

2023-04-27 16:27:33.418295: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-27 16:27:33.433507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-27 16:27:33.447742: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-27 16:27:33.462712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-27 16:27:33.475968: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-27 16:27:33.498146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-27 16:27:33.570703: I tensorflow/core/grappler/optimizers/cust

16/16 [==============================] - 3s 124ms/step - loss: 0.5259 - accuracy: 0.7580
[0.5259472727775574, 0.7580000162124634]


In [ ]:
# adversal validation